not working well 

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [5]:
def _calc_dodge(sub: pd.DataFrame) -> pd.Series:
    """
    Given all rows for ONE (year, quarter, question_number),
    return:
        • question_topic    first_topic of the question row
        • answer_topics     ';'-joined set of answer first_topics
        • dodge             1 if Q-topic is NOT in any answer-topics
    """
    q_rows = sub[sub["type"].str.lower() == "question"]
    q_topic = q_rows["first_topic"].iloc[0] if not q_rows.empty else ""

    ans_rows = sub[sub["type"].str.lower() == "answer"]
    ans_topics = set(ans_rows["first_topic"])
    dodge = int(q_topic not in ans_topics)

    return pd.Series(
        {
            "question_topic": q_topic,
            "answer_topics": ";".join(ans_topics) if ans_topics else "",
            "dodge": dodge,
        }
    )


def build_dodge_dashboard(
    folder: str = "qna",
    save_outputs: bool = True,
    min_sent_conf: float = 0.0,
):
    """
    Scan *folder*/**/*.xlsx  (expects columns like the screenshot).
    Returns
    -------
    dashboards : dict
        """
    
    dashboards = {}
    folder = Path(folder)

    for path in folder.glob("*.xlsx"):
        fn = path.stem
        df = pd.read_excel(path)
        df.columns = df.columns.str.lower()  # normalise headers

        # optional: drop low-confidence sentiment rows
        if "finbert_confid" in df.columns:
            df = df[df["finbert_confid"] >= min_sent_conf]

        # ----------------  add first_topic helper column  ----------------
        df["first_topic"] = (
            df["topics"]
            .fillna("")
            .str.split(",", n=1)
            .str[0]
        )

        # ----------------  aggregate per (Y,Q,question#)  ----------------
        pair_df = (
            df.groupby(["year", "quarter", "question_number"])
              .apply(_calc_dodge)
              .reset_index()
        )

        # ----------------  quarterly dodge-rate summary  ----------------
        summary_df = (
            pair_df.groupby(["year", "quarter"])["dodge"]
                   .mean()
                   .reset_index(name="dodge_rate")
        )

        dashboards[fn] = {"pair_df": pair_df, "summary_df": summary_df}

        # ----------------  optional Excel export  ----------------
        if save_outputs:
            out_dir = path.parent / f"{fn}_dodge"
            out_dir.mkdir(exist_ok=True)

            pair_df.to_excel(out_dir / f"{fn}_pair.xlsx", index=False)
            summary_df.to_excel(out_dir / f"{fn}_summary.xlsx", index=False)

    return dashboards

In [7]:
if __name__ == "__main__":
    dashboards = build_dodge_dashboard(folder="qna")

    # pick one file to inspect
    sample = next(iter(dashboards))  # first key
    print("Sample file:", sample)
    print(dashboards[sample]["pair_df"].head())
    print(dashboards[sample]["summary_df"].head())

C:\Users\dea33\AppData\Local\Temp\ipykernel_27424\4198509563.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["year", "quarter", "question_number"])


PermissionError: [Errno 13] Permission denied: 'qna\\qna_topics_UNY_dodge\\qna_topics_UNY_summary.xlsx'